In [ ]:
# ! pip install torch==1.9.0 --q
# ! pip install -U torchtext --q
# ! pip install -U torchnlp --q

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import spacy
import numpy as np

import random
import math
import time
import en_core_web_sm
import pandas as pd
from torch.utils.data import DataLoader

In [ ]:
%%bash
python -m spacy download en --q

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
s08 = pd.read_csv("/content/drive/MyDrive/END2.0/Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt", sep = '\t',encoding = "ISO-8859-1")
s09 = pd.read_csv("/content/drive/MyDrive/END2.0/Question_Answer_Dataset_v1.2/S09/question_answer_pairs.txt", sep = '\t',encoding = "ISO-8859-1")
s10 = pd.read_csv("/content/drive/MyDrive/END2.0/Question_Answer_Dataset_v1.2/S10/question_answer_pairs.txt", sep = '\t',encoding = "ISO-8859-1")

In [ ]:
df_qa = s08.append(s09).append(s10).reset_index()


In [ ]:
df_qa.head()

,index,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,data/set3/a4
1,1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,data/set3/a4
2,2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,data/set3/a4
3,3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,data/set3/a4
4,4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,data/set3/a4


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_qa, test_size=0.2)

In [ ]:
train.shape

(3198, 7)

In [ ]:
class CustomSQUADTextDataset(torch.utils.data.Dataset):
    def __init__(self, questions, answers):
        self.questions = [str(i) for i in list(questions)]
        self.answers = [str(i) for i in list(answers)] 
    def __len__(self):
        return len(self.answers)
    def __getitem__(self, idx):
        answer = self.answers[idx]
        question = self.questions[idx]
        sample = {"Question": question, "Answer": answer}
        return question, answer

In [ ]:
train_dataset = CustomSQUADTextDataset(train['Question'],train["Answer"])
test_dataset = CustomSQUADTextDataset(test['Question'],test["Answer"])

In [ ]:
train['Question']

1784                             What is Amedeo Avogadro?
910         With what party did Adams run for presidency?
1660                        What did Uruguay win in 1828?
1988     Did the modern hi-hat evolve from clash cymbals?
2053                        What are female foxes called?
                              ...                        
1931                What is Chinese language`s last name?
1140          What did Taylor and Fillmore disagree upon?
2580    Is Voltas legacy celebrated by a Temple on th...
3849    Is it possible to alter the word order to stre...
256          What years were Coolidge's two sons born in?
Name: Question, Length: 3198, dtype: object

In [ ]:
next(iter(train_dataset))

('What is Amedeo Avogadro?', 'A person.')

In [ ]:
# Display text and label.
print('\nFirst iteration of data set: ', next(iter(train_dataset)), '\n')
# Print how many items are in the data set
print('Length of data set: ', len(train_dataset), '\n')
# Print entire data set
print('Entire data set: ', list(DataLoader(train_dataset)), '\n')


First iteration of data set:  ('What is Amedeo Avogadro?', 'A person.') 

Length of data set:  3198 

Entire data set:  [[('What is Amedeo Avogadro?',), ('A person.',)], [('With what party did Adams run for presidency?',), ('The Federalist Party',)], [('What did Uruguay win in 1828?',), ('Its independence',)], [('Did the modern hi-hat evolve from clash cymbals?',), ('yes',)], [('What are female foxes called?',), ('vixens',)], [('When Becquerel discovered radioactivity, whose work did he investigate?',), ('Wilhelm Conrad Röntgen',)], [('Does Theodore Roosevelt have a brother?',), ('Yes',)], [('When was the Six Day War?',), ('1967',)], [('Is it possible that there are more than 350,000 species of beetles?',), ('yes',)], [('What was the Faraday effect first called?',), ('diamagnetism',)], [('What are the most common Cymbals?',), ('Hi-Hats, Crash, Splash, Ride, and China',)], [('Are turtle eggs leathery and soft?',), ('Yes',)], [('What was the Dead or Alive 4 fighting arena modeled after?

In [ ]:
train_questions = list(train["Question"])
train_answers = list(train["Answer"])


In [ ]:
train_list = train_questions + train_answers

In [ ]:
print(len(train_questions) + len(train_answers) == len(train_list))

True


In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
# from torchtext.datasets import Multi30k
from typing import Iterable, List

In [ ]:
token_transform = get_tokenizer('spacy', language='en')

In [ ]:
def yield_tokens(data_iter: Iterable ) -> List[str]:

    for data_sample in data_iter:
        yield token_transform(str(data_sample))

In [ ]:

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [ ]:
print(torch.__version__)
print(torchtext.__version__)

1.9.0+cu102
0.10.0


In [ ]:

vocab_transform = build_vocab_from_iterator(yield_tokens(train_list),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

In [ ]:
vocab_transform.vocab

In [ ]:

vocab_transform.set_default_index(UNK_IDX)

In [ ]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]), 
                      torch.tensor(token_ids), 
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}

text_transform = sequential_transforms(token_transform, #Tokenization
                                               vocab_transform, #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tesors
def collate_fn(batch):
    src_batch, src_len, tgt_batch = [], [], []
    for src_sample, tgt_sample in batch:
        # print(src_sample, tgt_sample)
        src_batch.append(text_transform(src_sample.rstrip("\n")))
        tgt_batch.append(text_transform(tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch,  tgt_batch

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N,value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)  # (N, value_len, heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N,query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        print("-"*20)
        print(positions.shape)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length= x.shape
        positions =  torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out


class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cpu",
        max_length=4,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        
        #src_mask = src.transpose(0, 1) == self.src_pad_idx
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len= trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        print ("-"*20)
        print("src_mask: ",src_mask.shape) 
        print("trg_mask: ", trg_mask.shape)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load_model = True
save_model = True

# Training hyperparameters
num_epochs = 10
learning_rate = 3e-4
BATCH_SIZE = 32

# Model hyperparameters
src_vocab_size = trg_vocab_size = len(vocab_transform) #len(german.vocab)
# trg_vocab_size =  len(vocab_transform[TGT_LANGUAGE]) #len(english.vocab)

src_pad_idx = PAD_IDX #english.vocab.stoi["<pad>"]
trg_pad_idx = PAD_IDX

model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(device)

In [ ]:
# DL_DS = DataLoader(train_dataset, batch_size=10, collate_fn=collate_fn)

# for i, batch in enumerate(DL_DS):
#     src,trg = batch
#     print(src,trg)
#     break

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Transformer(
  (encoder): Encoder(
    (word_embedding): Embedding(6156, 512)
    (position_embedding): Embedding(4, 512)
    (layers): ModuleList(
      (0): TransformerBlock(
        (attention): SelfAttention(
          (values): Linear(in_features=64, out_features=64, bias=False)
          (keys): Linear(in_features=64, out_features=64, bias=False)
          (queries): Linear(in_features=64, out_features=64, bias=False)
          (fc_out): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (feed_forward): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): ReLU()
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (dropout): Dropout(p=0, inplace=False)
      )
      (1): TransformerBlock(
        (attention): SelfAttention(
          (values): L

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 39,642,124 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [ ]:
# def train(model, optimizer, criterion, clip):
    
#     model.train()
    
#     epoch_loss = 0
#     train_dataloader = DataLoader(train_dataset, batch_size=10, collate_fn=collate_fn)

#     for src, trg  in train_dataloader: 
#         src = torch.transpose(src, 0,1).to(device) 
#         print(src.shape)
#         trg = torch.transpose(trg, 0,1).to(device)    
#         print(trg.shape)
#         #src = src.to(device)
#         #trg = trg.to(device)
#         #trg=trg[:,:-1]
#         #src_len = src_len.to(device)
        
              
#         optimizer.zero_grad()
        
#         output = model(src, trg)
        
#         #trg = [batch size, trg len]
#         #output = [batch size, trg len, output dim]
#         # output = output.reshape(-1, output.shape[2])
        
#         # trg = trg[:,1:].reshape(-1)
#         output_dim = output.shape[-1]
        
#         output = output[1:].reshape(-1, output_dim)
#         trg = trg[1:].reshape(-1)
              
#         loss = criterion(output, trg)
        
#         loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
#         optimizer.step()
        
#         epoch_loss += loss.item()
        
#     return epoch_loss / len(train_dataloader)


def train(model, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    for src, trg  in train_dataloader:
        # src = batch.Question[0]
        # trg = batch.Answer[0]
        src = torch.transpose(src, 0,1).to(device) 
        print(src.shape)
        trg = torch.transpose(trg, 0,1).to(device) 
        print("-"* 20)
        print("src.shape: ", src.shape)
        print("trg.shape: ", trg.shape)
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
        val_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
        for src,trg,_ in val_dataloader:    
            src=torch.transpose(src, 0,1).to(device) 
            trg=torch.transpose(trg, 0,1).to(device)
            #src_len = src_len.to(device)
            output = model(src, trg)
            
            #trg = [batch size, trg len]
            #output = [batch size, trg len, output dim]
            # output = output.reshape(-1, output.shape[2])
            
            # trg = trg[:,1:].reshape(-1)
            output_dim = output.shape[-1]
            
            output = output[1:].reshape(-1, output_dim)
            trg = trg[1:].reshape(-1)
            
            #trg = [batch size, trg len]
            #output = [batch size, trg len, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(val_dataloader)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model,  optimizer, criterion, CLIP)
    valid_loss = evaluate(model,  criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

torch.Size([32, 27])
--------------------
src.shape:  torch.Size([32, 27])
trg.shape:  torch.Size([32, 14])
--------------------
src_mask:  torch.Size([32, 1, 1, 27])
trg_mask:  torch.Size([32, 1, 14, 14])
--------------------
torch.Size([32, 27])


IndexError: ignored